fazer upload dados zip
unzip
upload nessa estrutura excluindo 5 imagens de cada tipo de test
https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/create-dataset-s3.html

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import io
import sagemaker.amazon.common as smac
import boto3
from sagemaker import get_execution_role
import sagemaker
import os

In [ ]:
### TODO how to use kaggle API

In [ ]:
### TODO traduzir comentarios e criar os markdowns

In [2]:
!pip install kaggle
#Oficial lab for lookoutforvision https://github.com/aws-samples/amazon-lookout-for-vision/blob/main/Amazon%20Lookout%20for%20Vision%20Lab.ipynb

     |████████████████████████████████| 59 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 1.0 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.10-py3-none-any.whl size=73269 sha256=c76803ab1b07b8b0afe2395340d2d9d1ca3f13b0995a198fdf9b6b3b2cc4f03b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1c/dd/dd/c493e6f981182c1411e288c553310f76e212bac3afbdac1294
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=e84849a935638a5b6cd473660ee19e56553095cfc112b12b75dcab8918b9b71c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
Successfully built kaggle python-slugify
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
# kaggle challenge -> https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product
!kaggle datasets download -d ravirajsinh45/real-life-industrial-dataset-of-casting-product

 98%|██████████████████████████████████████▏| 98.0M/100M [00:01<00:00, 88.3MB/s]
100%|████████████████████████████████████████| 100M/100M [00:01<00:00, 84.5MB/s]


In [5]:
!sudo yum install unzip -y

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
amzn-main                                                | 2.1 kB     00:00     
amzn-updates                                             | 3.8 kB     00:00     
Package unzip-6.0-4.10.amzn1.x86_64 already installed and latest version
Nothing to do


In [6]:
!unzip real-life-industrial-dataset-of-casting-product.zip

Archive:  real-life-industrial-dataset-of-casting-product.zip
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_0.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_100.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1015.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_102.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1046.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1053.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1055.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1077.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1087.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1091.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1104.jpeg  
  inflating: casting_512x512/casting_512x512/def_front/cast_def_0_1108.jpeg  
  infla

In [5]:
BUCKET_NAME = 'datasets-sagemaker-ml'
PROJECT_NAME = 'manufacturing'

In [ ]:
# TODO REMOVE BUCKET NAMES AND USE PARAMETER

In [9]:
# Using AWS CLI
!aws s3 cp ./casting_data/ s3://datasets-sagemaker-ml/ --recursive

upload: casting_data/casting_data/test/def_front/cast_def_0_112.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_112.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_1096.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_1096.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_1172.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_1172.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_1137.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_1137.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_1063.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_1063.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_108.jpeg to s3://datasets-sagemaker-ml/casting_data/test/def_front/cast_def_0_108.jpeg
upload: casting_data/casting_data/test/def_front/cast_def_0_1059.jpeg to s3://datasets-sagemaker-ml/casting_da

In [18]:
# Create manifest File

# Datetime for datetime generation and json to dump the JSON object
# to the corresponding files:
from datetime import datetime
import json

# Current date and time in manifest file format:
now = datetime.now()
dttm = now.strftime("%Y-%m-%dT%H:%M:%S.%f")

# The two datasets used: train and test
datasets = ["train", "test"]

# For each dataset...
for ds in datasets:
    # ...list the folder available (normal or anomaly).
    print(ds)
    folders = os.listdir("./casting_data/casting_data/{}".format(ds))
    # Then open the manifest file for this dataset...
    with open("{}.manifest".format(ds), "w") as f:
        for folder in folders:
            filecount=0
            print(folder)
            # ...and iterate through both folders by first listing
            # the corresponding files and setting the appropriate label
            # (as noted above: 1 = good, 0 = bad):
            files = os.listdir("./casting_data/casting_data/{}/{}".format(ds, folder))
            label = 1
            class_name = 'normal'
            if folder == "def_front":
                class_name = 'anomaly'
                label = 0
            # For each file in the folder...
            for file in files:
                # ...generate a manifest JSON object and save it to the manifest
                # file. Don't forget to add '/n' to generate a new line:
                manifest = {
                  "source-ref": "s3://{}/{}/{}/{}/{}".format(BUCKET_NAME, 'casting_data', ds, folder, file),
                  "auto-label": label,
                  "auto-label-metadata": {
                    "confidence": 1,
                    "job-name": "labeling-job/auto-label",
                    "class-name": class_name,
                    "human-annotated": "yes",
                    "creation-date": dttm,
                    "type": "groundtruth/image-classification"
                  }
                }
                f.write(json.dumps(manifest)+"\n")

train
def_front
ok_front
test
def_front
ok_front


In [19]:
# Upload manifest files to S3 bucket:
!aws s3 cp train.manifest s3://datasets-sagemaker-ml/train.manifest
!aws s3 cp test.manifest s3://datasets-sagemaker-ml/test.manifest

upload: ./train.manifest to s3://datasets-sagemaker-ml/train.manifest
upload: ./test.manifest to s3://datasets-sagemaker-ml/test.manifest


In [2]:
client = boto3.Session().client('lookoutvision')

In [13]:
#Creating project
print('Creating project:' + PROJECT_NAME)
response=client.create_project(ProjectName=PROJECT_NAME)
print('project ARN: ' + response['ProjectMetadata']['ProjectArn'])
print('Done!')

Creating project:manufacturing
project ARN: arn:aws:lookoutvision:us-east-1:475414269301:project/manufacturing
Done!


https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lookoutvision.html#LookoutforVision.Client.create_dataset

In [20]:
#Creating training dataset
dataset_type ='train'
manifest_file = 'train.manifest'

print('Creating dataset...')
dataset=json.loads('{ "GroundTruthManifest": { "S3Object": { "Bucket": "' + BUCKET_NAME + '", "Key": "'+ manifest_file + '" } } }')

response=client.create_dataset(ProjectName=PROJECT_NAME, DatasetType=dataset_type, DatasetSource=dataset)
print('Dataset Status: ' + response['DatasetMetadata']['Status'])
print('Dataset Status Message: ' + response['DatasetMetadata']['StatusMessage'])
print('Dataset Type: ' + response['DatasetMetadata']['DatasetType'])
print('Done!')

Creating dataset...
Dataset Status: CREATE_IN_PROGRESS
Dataset Status Message: The dataset is creating.
Dataset Type: train
Done!


In [21]:

#Creating test dataset
dataset_type ='test'
manifest_file = 'test.manifest'

print('Creating dataset...')
dataset=json.loads('{ "GroundTruthManifest": { "S3Object": { "Bucket": "' + BUCKET_NAME + '", "Key": "'+ manifest_file + '" } } }')

response=client.create_dataset(ProjectName=PROJECT_NAME, DatasetType=dataset_type, DatasetSource=dataset)
print('Dataset Status: ' + response['DatasetMetadata']['Status'])
print('Dataset Status Message: ' + response['DatasetMetadata']['StatusMessage'])
print('Dataset Type: ' + response['DatasetMetadata']['DatasetType'])
print('Done!')

Creating dataset...
Dataset Status: CREATE_IN_PROGRESS
Dataset Status Message: The dataset is creating.
Dataset Type: test
Done!


In [22]:
#Creating/training model
output_bucket = BUCKET_NAME
output_folder = 'model/'

    
print('Creating model...')
output_config=dataset=json.loads('{ "S3Location": { "Bucket": "' + output_bucket + '", "Prefix": "'+ output_folder + '" } } ')

response=client.create_model(ProjectName=PROJECT_NAME, OutputConfig=output_config)
print('ARN: ' + response['ModelMetadata']['ModelArn'])
print('Version: ' + response['ModelMetadata']['ModelVersion'])
print('Status: ' + response['ModelMetadata']['Status'])
print('Message: ' + response['ModelMetadata']['StatusMessage'])
print('Done!')

Creating model...
ARN: arn:aws:lookoutvision:us-east-1:475414269301:model/manufacturing/2
Version: 2
Status: TRAINING
Message: The model is being trained.
Done!


In [24]:
# Check if the model completed the training
import time

while client.describe_model(ProjectName=PROJECT_NAME,ModelVersion='2')['ModelDescription']['Status']!='TRAINED':
    print('.',end='');time.sleep(5);
print('Done!')

TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.TRAINING
.

In [8]:
# Hosting trained model
model_version='2'
min_inference_units=1 
    
print('Starting model version ' + model_version  + ' for project ' + PROJECT_NAME )
response=client.start_model(ProjectName=PROJECT_NAME,
    ModelVersion=model_version,
    MinInferenceUnits=min_inference_units)
print('Status: ' + response['Status'])

Starting model version 2 for project manufacturing
Status: STARTING_HOSTING


In [10]:
import time
# Waiting for model host to complete
while client.describe_model(ProjectName=PROJECT_NAME,ModelVersion='2')['ModelDescription']['Status']!='HOSTED':
    print('.',end='');time.sleep(5);
print('Done!')

.............................................................Done!


In [11]:
'''
I Created a Folder called extra-images and put some images from the test dataset to validate the model (colocar a criação do extra-images no inicio)
'''

'\nI Created a Folder called extra-images and put some images from the test dataset to validate the model\n'

In [12]:
# Make Predictions

#Picking an anamolous image from the extra images
photo='extra-images/cast_def_0_1635.jpeg'
model_version='2'
    
with open(photo, 'rb') as image:
    response = client.detect_anomalies(ProjectName=PROJECT_NAME, 
    ContentType='image/jpeg',
    Body=image.read(),
    ModelVersion=model_version)
print ('Anomalous?: ' + str(response['DetectAnomalyResult']['IsAnomalous']))
print ('Confidence: ' + str(response['DetectAnomalyResult']['Confidence']))

Anomalous?: True
Confidence: 0.9999916553497314


In [15]:
photo='extra-images/cast_ok_0_45.jpeg'
model_version='2'
    
with open(photo, 'rb') as image:
    response=client.detect_anomalies(ProjectName=PROJECT_NAME, 
    ContentType='image/jpeg',
    Body=image.read(),
    ModelVersion=model_version)
print ('Anomalous?: ' + str(response['DetectAnomalyResult']['IsAnomalous']))
print ('Confidence: ' + str(response['DetectAnomalyResult']['Confidence']))

Anomalous?: False
Confidence: 0.9839045405387878


In [ ]:
#If you are not using the model, stop to save costs!
model_version='2'

print('Stopping model version ' + model_version  + ' for project ' + PROJECT_NAME )
response=client.stop_model(ProjectName=project,
    ModelVersion=model_version)
print('Status: ' + response['Status'])